# photometry preprocessing 

This notebook contains allows exploration of different preprocessing options for photometry data. There are various different ways to preprocess this kind of data, so the idea behind this notebook is to act as an easy-to-use resource for trying various approaches out and visualising the results.

The contents of this notebook are heavily inspired by Thomas Akam's code [here](https://github.com/ThomasAkam/photometry_preprocessing/blob/master/Photometry%20data%20preprocessing.ipynb) as well as discussions with Marta Blanco Pozo and Luke Coddington. However, if you come across any errors or have any questions please message me at grimal@janelia.hhmi.org.

The main steps for the preprocessing are as follows: 
1. 

The data used in this notebook were recorded from the NACc core of a mouse during a freely moving reward-based task. [pyPhotometry](https://pyphotometry.readthedocs.io/en/latest/) acquisition hardware and software was used in two colour time division mode, with GCaMP8m as the calcium indicator and tdTomato as movement control. 


Start by importing python modules needed and setting up figure parameters.

In [4]:
import os
import numpy as  np
import pylab as plt
from scipy.signal import medfilt, butter, filtfilt
from scipy.stats import linregress
from scipy.optimize import curve_fit, minimize

plt.rcParams['figure.figsize'] = [14, 12] # Make default figure size larger.
plt.rcParams['axes.xmargin'] = 0          # Make default margin on x axis zero.

Import the pyPhotometry data import module.

In [ ]:
from import_photometry import import_ppd